# Kerakli kutubxonalarni chaqiramiz

In [2]:
import numpy as np
import pandas as pd

# dataset ni yuklaymiz

In [11]:
df = pd.read_csv(r"C:\Users\bunyo\OneDrive\Desktop\AI_Course\ModularProgramProjects\FirstWeekProject\data\raw_data\wind_dataset.csv")

In [12]:
df.sample(6)

,DATE,WIND,IND,RAIN,IND.1,T.MAX,IND.2,T.MIN,T.MIN.G
5810,1976-11-28,21.84,0,5.0,0.0,8.5,0.0,4.5,3.6
196,1961-07-16,13.13,0,0.1,0.0,15.6,0.0,11.0,10.1
4938,1974-07-10,11.54,0,0.3,0.0,21.3,0.0,13.3,10.2
1544,1965-03-25,17.12,0,6.8,4.0,NaN,4.0,NaN,0.0
2537,1967-12-13,6.67,4,0.0,0.0,8.6,0.0,6.4,3.4
1578,1965-04-28,8.33,0,0.3,0.0,10.3,0.0,5.4,4.6


# DATE ustuni keraksizligi uchun tashab yubaramiz

In [13]:
df = df.drop('DATE', axis=1)

In [14]:
df

,WIND,IND,RAIN,IND.1,T.MAX,IND.2,T.MIN,T.MIN.G
0,13.67,0,0.2,0.0,9.5,0.0,3.7,-1.0
1,11.50,0,5.1,0.0,7.2,0.0,4.2,1.1
2,11.25,0,0.4,0.0,5.5,0.0,0.5,-0.5
3,8.63,0,0.2,0.0,5.6,0.0,0.4,-3.2
4,11.92,0,10.4,0.0,7.2,1.0,-1.5,-7.5
...,...,...,...,...,...,...,...,...
6569,14.46,0,16.8,0.0,9.8,0.0,4.0,0.0
6570,14.33,0,16.0,0.0,9.1,0.0,8.5,8.0
6571,19.17,0,14.7,0.0,5.0,0.0,3.5,3.2
6572,18.08,0,4.9,0.0,2.9,0.0,0.3,-0.5
